In [ ]:
!pip install cornac
!pip3 install dgl

In [ ]:
import pandas as pd
import time
import datetime
import random

In [ ]:
import cornac
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality
from cornac.data.text import BaseTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/dataset/data_merged.csv')

In [ ]:
convert_timestamp = lambda x: time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple())

df['timestamp'] = df['Date'].apply(convert_timestamp)

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

In [ ]:
df = df[df.IDuser.map(df.IDuser.value_counts()) > 4]

In [ ]:
ratings = df.apply(lambda row: (row.IDuser, row.IDhotel, row.RatingHotel), axis=1).tolist()

In [ ]:
hotels = df.apply(lambda row: (row.Overview), axis=1).tolist()

In [ ]:
item_ids = df.apply(lambda row: (row.IDhotel), axis=1).tolist()

In [ ]:
item_text_modality = TextModality(
    corpus=hotels,
    ids=item_ids,
    tokenizer=BaseTokenizer(stop_words="english"),
    max_vocab=8000,
    max_doc_freq=0.5,
)

In [ ]:
ratio_split = RatioSplit(
    data=ratings,
    test_size=0.2,
    exclude_unknowns=True,
    item_text=item_text_modality,
    verbose=True,
    seed=42,
    rating_threshold=0.5,
)

rating_threshold = 0.5
exclude_unknowns = True
---
Training data:
Number of users = 247
Number of items = 239
Number of ratings = 1610
Max rating = 9.8
Min rating = 0.0
Global mean = 8.4
---
Test data:
Number of users = 247
Number of items = 239
Number of ratings = 412
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 247
Total items = 239


/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 81 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 4 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [ ]:
# Use Recall@10 for evaluation
rec_10 = cornac.metrics.Recall(k=10)
rec_5 = cornac.metrics.Recall(k=5)
pre_10 = cornac.metrics.Precision(k=10)
pre_5 = cornac.metrics.Precision(k=5)

In [ ]:
metrics = [rec_5, pre_5, rec_10, pre_10]

In [ ]:
# Instantiate CTR model
ctr = cornac.models.CTR(k=50, max_iter=50, lambda_v=1)

In [ ]:
# Instantiate HTF model
hft = cornac.models.HFT(
    k=10,
    max_iter=40,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=5000,
    seed=123,
)

In [ ]:
# Instantiate ConvMF model
convmf = cornac.models.ConvMF(n_epochs=5, verbose=True, seed=42)

In [ ]:
models = [convmf, ctr, hft]

In [ ]:
# Put everything together into an experiment and run it
cornac.Experiment(eval_method=ratio_split, models=models, metrics=metrics).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 5362340.39188 Elapsed: 21.9938s Converge: 536234039188163131050315578969377922175835663188348633088.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 1276733.08850 Elapsed: 23.1226s Converge: 0.761907 
Epoch: 3/5


...


[HFT] Training started!


  0%|          | 0/40 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Ranking:   0%|          | 0/198 [00:00<?, ?it/s]


TEST:
...
       | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
------ + ------------ + ----------- + --------- + -------- + --------- + --------
ConvMF |       0.0152 |      0.0162 |    0.0689 |   0.0302 |  113.0675 |   0.3985
CTR    |       0.0374 |      0.0586 |    0.1735 |   0.1375 |   12.8960 |   0.2280
HFT    |       0.0010 |      0.0000 |    0.0035 |   0.0000 |   42.1645 |   0.2057

